# Задача 10. Graph Convolutional Network

* **Дедлайн**: до экзамена
* Основной полный балл: 5
* Максимум баллов: 10


## Задача

- [x] Найти графовый набор данных для решения задачи предсказания (классификация вершин, обнаружение сообществ и т.д.).
- [x] Использовать несколько слоев `GCNConv` из библиотеки `PyG` для построения GCN модели.
- [x] Обучить полученную модель, подобрать гиперпараметры (например, learning rate) на валидационной выборке, и оценить качество предсказания на тестовой выборке.
- [x] (+5 баллов) Также представить самостоятельную реализацию слоя `GCNConv`, используя матричные операции. Повторить обучение с собственными слоями и сравнить результаты.


**Выполнил**: Азим Мурадов

**Университет**: СПбГУ

**Группа**: 22.Б11-мм

В качестве датасета будет использован **MUTAG** датасет. Это классический датасет для задач бинарной классификации графов, содержащий информацию о химических соединениях.

- **Тип графов**: неориентированные графы молекул
- **Вершины**: атомы (без атомов водорода)
- **Рёбра**: химические связи между атомами
- **Метки графов (классы)**:
    - +1 — мутагенное соединение (может вызывать мутации ДНК)
    - -1 — не мутагенное
- **Кол-во графов**: 188
- **Средний кол-во вершин**: \~17.93 вершин
- **Средний кол-во рёбер**: \~19.79 рёбер

**Источник**:

- Датасет входит в TUDataset Collection: https://chrsmrrs.github.io/datasets/
- Cсылка на архив датасета: https://chrsmrrs.github.io/datasets/MUTAG.zip
- Первоначальная публикация:
_Debnath et al., “Structure-activity relationship of mutagenic aromatic and heteroaromatic nitro compounds”, Journal of Chemical Information and Computer Sciences, 1991._

In [1]:
import torch
import torch.nn as nn

In [2]:
from torch_geometric.datasets import TUDataset
from pathlib import Path


dataset = TUDataset(root=(Path("..") / "data").resolve(), name="MUTAG")
print(dataset)
print(dataset[0])

num_features = dataset.num_features
hidden_dim = 32
num_classes = dataset.num_classes

print(f"Number of features: {num_features}, Number of classes: {num_classes}")

MUTAG(188)
Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of features: 7, Number of classes: 2


Processing...
Done!


In [3]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader


# Split to train/val/test
idx = list(range(len(dataset)))
temp_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=42)
train_idx, val_idx = train_test_split(temp_idx, test_size=0.2, random_state=42)
train_dataset = dataset[train_idx]
val_dataset = dataset[val_idx]
test_dataset = dataset[test_idx]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Построим GCN модель

In [4]:
from torch_geometric.nn import global_mean_pool
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, gcn_conv, num_features, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = gcn_conv(num_features, hidden_dim)
        self.conv2 = gcn_conv(hidden_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return x

## Обучим полученную модель используя `GCNConv`, обучение проведем подбирая гиперпараметры. Оценим качество предсказания.

In [5]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x.float(), data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

In [6]:
def test(model, loader, device):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x.float(), data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

In [7]:
def grid_search(layer_class, num_features, num_classes, train_loader, val_loader, device):
    best_val_acc = 0
    best_params = None
    best_model_state = None
    for hidden_dim in [16, 32, 64]:
        for lr in [0.01, 0.005, 0.001]:
            model = GCN(layer_class, num_features, hidden_dim, num_classes).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            for _ in range(0, 100):
                train(model, train_loader, optimizer, device)
            val_acc = test(model, val_loader, device)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_params = {"hidden_dim": hidden_dim, "lr": lr}
                best_model_state = model.state_dict()
    return best_params, best_val_acc, best_model_state

In [8]:
from torch_geometric.nn import GCNConv


print("[GCNConv] Grid search:")
best_params, best_val_acc, best_model_state = grid_search(
    GCNConv, num_features, num_classes, train_loader, val_loader, device
)

print(f"[GCNConv] Best params: {best_params}")
print(f"[GCNConv] Validation Accuracy (with best params): {best_val_acc:.4f}")
model = GCN(GCNConv, num_features, best_params["hidden_dim"], num_classes).to(device)
model.load_state_dict(best_model_state)

gcn_conv_test_acc = test(model, test_loader, device)
print(f"[GCNConv] Test Accuracy (with best params): {gcn_conv_test_acc:.4f}")

[GCNConv] Grid search:
[GCNConv] Best params: {'hidden_dim': 16, 'lr': 0.01}
[GCNConv] Validation Accuracy (with best params): 0.6667
[GCNConv] Test Accuracy (with best params): 0.8158


## Представим самостоятельную реализацию слоя `GCNConv`, используя матричные операции.

In [9]:
class MyGCNConv(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, edge_index):
        num_nodes = x.size(0)
        edge_index = edge_index
        adj = torch.zeros((num_nodes, num_nodes), device=x.device)
        adj[edge_index[0], edge_index[1]] = 1
        adj = adj + torch.eye(num_nodes, device=x.device)
        deg = adj.sum(dim=1)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float("inf")] = 0
        norm_adj = deg_inv_sqrt.unsqueeze(1) * adj * deg_inv_sqrt.unsqueeze(0)
        x = self.linear(x)
        x = norm_adj @ x
        return x

## Обучим полученную модель используя `MyGCNConv`, обучение проведем подбирая гиперпараметры. Оценим качество предсказания.

In [10]:
print("[MyGCNConv] Grid search:")
best_params, best_val_acc, best_model_state = grid_search(
    MyGCNConv, num_features, num_classes, train_loader, val_loader, device
)

print(f"[MyGCNConv] Best params: {best_params}")
print(f"[MyGCNConv] Validation Accuracy (with best params): {best_val_acc:.4f}")
model = GCN(MyGCNConv, num_features, best_params["hidden_dim"], num_classes).to(device)
model.load_state_dict(best_model_state)

my_gcn_conv_test_acc = test(model, test_loader, device)
print(f"[MyGCNConv] Test Accuracy (with best params): {my_gcn_conv_test_acc:.4f}")

[MyGCNConv] Grid search:
[MyGCNConv] Best params: {'hidden_dim': 16, 'lr': 0.01}
[MyGCNConv] Validation Accuracy (with best params): 0.6667
[MyGCNConv] Test Accuracy (with best params): 0.8158


In [11]:
print(f"[GCNConv] Test Accuracy: {gcn_conv_test_acc:.4f}")
print(f"[MyGCNConv] Test Accuracy: {my_gcn_conv_test_acc:.4f}")

[GCNConv] Test Accuracy: 0.8158
[MyGCNConv] Test Accuracy: 0.8158
